In [1]:
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data.dataloader import DataLoader
import torch 
import numpy as np

mean=[0.49139968, 0.48215827 ,0.44653124]
std=[0.24703233, 0.24348505, 0.26158768]
transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize(mean=mean,std=std)]
                              )
ds = CIFAR10(root='/tmp', download=True, transform=transform)

dl = DataLoader(ds,
                batch_size=100,
                pin_memory=True,
                num_workers=2,
                )

/Users/rvx/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Files already downloaded and verified


In [2]:
cal_ds = []
cal_lbl = []

calibration_length = 100
for batch in dl:
    if len(cal_lbl) > calibration_length:
        break
    images, labels = batch
    cal_ds.extend(images.cpu().numpy().tolist())
    cal_lbl.extend(labels.cpu().numpy().tolist())

calibration_samples = np.array(cal_ds, dtype=np.float32)[:calibration_length]
calibration_labels = np.expand_dims(np.array(cal_lbl)[:calibration_length], axis=1)

/Users/rvx/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/rvx/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
from torchto.converter import Converter

2024-01-15 12:04:16.277338: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/rvx/Library/Python/3.9/lib/python/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
# CUB
#calibration_samples = np.load('../../cub_samples_cab.npy')
#calibration_labels = np.load('../../cub_labels_cab.npy')

In [5]:
#model = torch.load('../../zico_bc.pth', map_location='cpu')

In [6]:
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=True)
dummy_input = torch.rand(1,3,32,32)

for conv_type in ['ONNX']:
    print('-'*30)
    print(conv_type)
    converter = Converter(conv_type,
                          calibration_samples=calibration_samples,
                          calibration_labels=calibration_labels,
                          dummy_input=dummy_input, 
                          verbose=False)
    converted_model = converter(model, dir='dummy', name='dummy')



Using cache found in /Users/rvx/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


------------------------------
ONNX
================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [7]:
import onnx
dummy_input = torch.rand(1,3,32,32)
model = onnx.load('dummy/dummy.onnx')
for conv_type in ['TFLITE']:
    print('-'*30)
    print(conv_type)
    converter = Converter(conv_type,
                          calibration_samples=calibration_samples,
                          calibration_labels=calibration_labels,
                          dummy_input=dummy_input, 
                          verbose=False)
    converted_model = converter(model, dir='dummy', name='dummy')

------------------------------
TFLITE
INFO:tensorflow:Assets written to: dummy/dummy/assets


INFO:tensorflow:Assets written to: dummy/dummy/assets
2024-01-15 12:05:16.319022: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-01-15 12:05:16.319051: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-01-15 12:05:16.320763: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: dummy/dummy
2024-01-15 12:05:16.323561: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-01-15 12:05:16.323576: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: dummy/dummy
2024-01-15 12:05:16.328872: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2024-01-15 12:05:16.330066: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-01-15 12:05:16.386791: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedMo